<a href="https://colab.research.google.com/github/YKG-123/Fantasy_football_ML_Random_forest_Regressor/blob/main/TE_preseason_postable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')
QB_2025=pd.read_csv('/content/drive/MyDrive/TE_2025.csv')
QB_2025.rename(columns={"adp":"adp_for_model"})
QB_2025.rename(columns={"adp_for_rank": "adp_true"}, inplace=True)
QB_2025=QB_2025.reset_index(drop=True)
QB_2025.drop(columns=["adp_true"], inplace=True)
QB_2025.rename(columns={"adp_true_y": "adp"}, inplace=True)

Mounted at /content/drive


In [3]:
QB_2025.columns

Index(['Unnamed: 0', 'adp', 'player_name', 'fantasy_points_ppr_prev',
       'predicted_target_season', 'touches_prev', 'adp_rank',
       'predicted_rank'],
      dtype='object')

In [4]:

# --- 1. Ensure column uniqueness ---
QB_2025 = QB_2025.loc[:, ~QB_2025.columns.duplicated()]

# --- 2. Fill missing ADP with sentinel ---
QB_2025["adp"] = QB_2025["adp"].fillna(200)

# --- 3. Filter out undrafted ---
QB_2025 = QB_2025[QB_2025["adp"] != 200]  # keep only drafted players

# --- 4. Compute ranks ---
QB_2025["predicted_rank"] = QB_2025["predicted_target_season"].rank(ascending=False)
QB_2025["adp_rank"]       = QB_2025["adp"].rank(ascending=True)

# --- 5. ADP minus model prediction gap ---
QB_2025["adp-predictions"] = QB_2025["adp_rank"] - QB_2025["predicted_rank"]

# --- 6. Sort so biggest difference appears first ---
QB_2025 = QB_2025.sort_values(by="predicted_rank", ascending=True).reset_index(drop=True)
QB_2025["adp"] = QB_2025["adp"].round(2)
# --- 7. Preview top 75 WRs ---
QB_2025[["player_name", "adp", "predicted_rank", "adp_rank", "adp-predictions","predicted_target_season"]].head(100)



,player_name,adp,predicted_rank,adp_rank,adp-predictions,predicted_target_season
0,Brock Bowers,26.8,1.0,1.0,0.0,12.150662
1,Trey McBride,31.1,2.0,2.0,0.0,11.726042
2,George Kittle,43.1,3.0,3.0,0.0,9.745512
3,Travis Kelce,67.7,4.0,5.0,1.0,9.134817
4,Sam LaPorta,63.2,5.0,4.0,-1.0,9.092665
5,Tucker Kraft,108.9,6.0,10.0,4.0,8.495536
6,Zach Ertz,143.7,7.0,16.0,9.0,8.045169
7,Colston Loveland,141.8,8.0,15.0,7.0,7.927975
8,Dalton Kincaid,133.6,9.0,14.0,5.0,7.927336
9,Isaiah Likely,155.2,10.0,19.0,9.0,7.718799
